## Using SyntaxIgnoreTagger in pre-processing for syntactic analysis

Some parts of a text may be difficult to analyse syntactically. For instance, enumerations of sports results, and content in parentheses (references and short remarks) may exhibit only very little linguistic structure suitable for analysis. So, you may want to skip the syntactic analysis on such parts of the text.

In order to detect parts of text that should be ignored by the syntactic analysis, EstNLTK has a special tagger called SyntaxIgnoreTagger.

In [1]:
from estnltk.taggers.syntax_preprocessing.syntax_ignore_tagger import SyntaxIgnoreTagger
syntax_ignore_tagger = SyntaxIgnoreTagger()
syntax_ignore_tagger

SyntaxIgnoreTagger(input_layers=['words', 'sentences'], output_layer=syntax_ignore, output_attributes=('type',), allow_loose_match=True, ignore_parenthesized_num=True, ignore_parenthesized_num_greedy=True, ignore_parenthesized_ref=True, ignore_parenthesized_title_words=True, ignore_parenthesized_short_char_sequences=True, ignore_consecutive_parenthesized_sentences=True, ignore_consecutive_enum_ucase_num_sentences=True, ignore_sentences_consisting_of_numbers=True, ignore_sentences_starting_with_time=True, ignore_sentences_with_comma_separated_num_name_lists=True, ignore_brackets=True, depends_on=['words', 'sentences'], layer_name=syntax_ignore, attributes=('type',))

SyntaxIgnoreTagger requires that 'words' and 'sentences' have been annotated in text. It uses these layers to detect snippets of text (not necessarily full sentences) which should be ignored during the syntactic analysis. Detection patterns are partly based on the patterns used in pre-processing modules of EstSyntax, available [here](https://github.com/kristiinavaik/ettenten-eeltootlus) and [here](https://github.com/EstSyntax/preprocessing-module).

Similar detection patterns have been grouped together. Upon the initialization of the tagger, flags can be used to switch these groups off (by default, all flags have been switched on). In following, we will give a minute introduction on the flags, and types of ignorable text snippets the corresponding patterns aim to extract.

### Patterns for detecting ignore content inside sentences

#### Flag `ignore_brackets`

Content inside square brackets will be ignored.

In [2]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Nurksulgudes tuuakse materjali viitekirje järjekorranumber kirjanduse loetelus ja leheküljed , nt [9: 5] või [9 lk 5], aga internetimaterjalil lihtsalt viitekirje, nt [7]')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_brackets=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name=syntax_ignore, spans=SL[ES[Span(start=98, end=99, text='['),
Span(start=99, end=100, text='9'),
Span(start=100, end=101, text=':'),
Span(start=102, end=103, text='5'),
Span(start=103, end=104, text=']')],
ES[Span(start=109, end=110, text='['),
Span(start=110, end=111, text='9'),
Span(start=112, end=114, text='lk'),
Span(start=115, end=116, text='5'),
Span(start=116, end=117, text=']')],
ES[Span(start=167, end=168, text='['),
Span(start=168, end=169, text='7'),
Span(start=169, end=170, text=']')]])

#### Flag `ignore_parenthesized_ref`

Parenthesized content which looks like a reference (e.g. contains titlecased words, and date information) will be ignored.

In [3]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Tutvustamisele tulevad Jan Kausi romaan “Koju” (Tuum 2012) ning Ülo Pikkovi romaan “Vana prints” (Varrak 2012). Temaatikaga seondub veel teinegi äsja Postimehes ilmunud jutt (Priit Pullerits «Džiibi kaitseks», PM 30.07.2010).')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_parenthesized_ref=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name=syntax_ignore, spans=SL[ES[Span(start=47, end=48, text='('),
Span(start=48, end=52, text='Tuum'),
Span(start=53, end=57, text='2012'),
Span(start=57, end=58, text=')')],
ES[Span(start=97, end=98, text='('),
Span(start=98, end=104, text='Varrak'),
Span(start=105, end=109, text='2012'),
Span(start=109, end=110, text=')')],
ES[Span(start=174, end=175, text='('),
Span(start=175, end=180, text='Priit'),
Span(start=181, end=190, text='Pullerits'),
Span(start=191, end=192, text='«'),
Span(start=192, end=198, text='Džiibi'),
Span(start=199, end=207, text='kaitseks'),
Span(start=207, end=209, text='»,'),
Span(start=210, end=212, text='PM'),
Span(start=213, end=223, text='30.07.2010'),
Span(start=223, end=224, text=')')]])

#### Flag `ignore_parenthesized_short_char_sequences`

Parenthesized short sequences of tokens (up to 4 tokens), each of which also has as short length (up to 4 characters), will be ignored.

In [4]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Eesti judokate võistlus jäi laupäeval lühikeseks , nii Joel Rothberg ( -66 kg ) kui ka Renee Villemson ( -73 kg ) võidurõõmu maitsta ei saanud .')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_parenthesized_short_char_sequences=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name=syntax_ignore, spans=SL[ES[Span(start=69, end=70, text='('),
Span(start=71, end=74, text='-66'),
Span(start=75, end=77, text='kg'),
Span(start=78, end=79, text=')')],
ES[Span(start=103, end=104, text='('),
Span(start=105, end=108, text='-73'),
Span(start=109, end=111, text='kg'),
Span(start=112, end=113, text=')')]])

#### Flag `ignore_parenthesized_title_words`

Parenthesized 1-2 titlecase words (which may be comma-separated) will be ignored.

In [5]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Neidude 5 km klassikat võitis Lina Andersson ( Rootsi ) Pirjo Mannineni ( Soome ) ja Karin Holmbergi ( Rootsi ) ees .')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_parenthesized_title_words=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name=syntax_ignore, spans=SL[ES[Span(start=45, end=46, text='('),
Span(start=47, end=53, text='Rootsi'),
Span(start=54, end=55, text=')')],
ES[Span(start=72, end=73, text='('),
Span(start=74, end=79, text='Soome'),
Span(start=80, end=81, text=')')],
ES[Span(start=101, end=102, text='('),
Span(start=103, end=109, text='Rootsi'),
Span(start=110, end=111, text=')')]])

#### Flag `ignore_parenthesized_num`

Parenthesized numerics (such as dates, date ranges, number sequences) will be ignored.

In [6]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Klubi sai kuus korda Inglismaa meistriks (1976, 1977, 1979, 1980, 1982, 1983). Tallinna ( 21.-22. mai ) , Haapsalu ( 2.-3. juuli ) ja Liivimaa ( 30.-31. juuli ) rallidel on vähemalt see probleem lahendatud .')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_parenthesized_num=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name=syntax_ignore, spans=SL[ES[Span(start=41, end=42, text='('),
Span(start=42, end=46, text='1976'),
Span(start=46, end=47, text=','),
Span(start=48, end=52, text='1977'),
Span(start=52, end=53, text=','),
Span(start=54, end=58, text='1979'),
Span(start=58, end=59, text=','),
Span(start=60, end=64, text='1980'),
Span(start=64, end=65, text=','),
Span(start=66, end=70, text='1982'),
Span(start=70, end=71, text=','),
Span(start=72, end=76, text='1983'),
Span(start=76, end=77, text=')')],
ES[Span(start=88, end=89, text='('),
Span(start=90, end=93, text='21.'),
Span(start=93, end=94, text='-'),
Span(start=94, end=97, text='22.'),
Span(start=98, end=101, text='mai'),
Span(start=102, end=103, text=')')],
ES[Span(start=115, end=116, text='('),
Span(start=117, end=119, text='2.'),
Span(start=119, end=120, text='-'),
Span(start=120, end=122, text='3.'),
Span(start=123, end=128, text='juuli'),
Span(start=129, end=130, text=')')],
ES[Span(start=143, end=144, text='('),
Span(start=145, end=148, text='30.'),
Span(start=148, end=149, text='-'),
Span(start=149, end=152, text='31.'),
Span(start=153, end=158, text='juuli'),
Span(start=159, end=160, text=')')]])

#### Flag `ignore_parenthesized_num_greedy`

Applies greedy parenthesized numeric content detection patterns: if there is at least one number inside parentheses, but there cannot be found at least 3 consecutive lowercase words, then the whole content inside parentheses will be marked as to be ignored.

In [7]:
# Create text and preprocess
from estnltk.text import Text
text = Text('Näited: A (300 000 000 m/sek), B ( naised 5 km , mehed 7,5 km ) ning C (meie puhul 165/80) .')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_parenthesized_num_greedy=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name=syntax_ignore, spans=SL[ES[Span(start=10, end=11, text='('),
Span(start=11, end=22, text='300 000 000'),
Span(start=23, end=28, text='m/sek'),
Span(start=28, end=29, text=')')],
ES[Span(start=33, end=34, text='('),
Span(start=35, end=41, text='naised'),
Span(start=42, end=43, text='5'),
Span(start=44, end=46, text='km'),
Span(start=47, end=48, text=','),
Span(start=49, end=54, text='mehed'),
Span(start=55, end=58, text='7,5'),
Span(start=59, end=61, text='km'),
Span(start=62, end=63, text=')')],
ES[Span(start=71, end=72, text='('),
Span(start=72, end=76, text='meie'),
Span(start=77, end=82, text='puhul'),
Span(start=83, end=86, text='165'),
Span(start=86, end=87, text='/'),
Span(start=87, end=89, text='80'),
Span(start=89, end=90, text=')')]])

Note that this pattern group also covers content detected by some other `ignore_parenthesized_*` patterns. So, if you want to turn other patterns off, you may also want to turn off this pattern.

#### Flag `allow_loose_match`

If `True` (default setting), then an ignore text snippet may consume words without matching exactly with their boundaries (e.g. ignore snippet's start does not have to match word's start). If `False`, then an ignore text snippet must match exactly with word boundaries: it must start where a word starts, and end where a word ends.

### Patterns for ignoring full sentences

#### Flag  `ignore_sentences_starting_with_time`

Sentences starting with a date range (e.g. a time schedule of a seminar, or a TV program) will be ignored.

In [8]:
# Create text and preprocess
from estnltk.text import Text
text = Text('''
12.05 - 12.35 "Õnne 13" (1. osa)
12.35 - 13.05 "Õnne 13" (1. osa kordus)
13.05 - 13.35 "Õnne 13" (2. osa)
''')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_sentences_starting_with_time=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name=syntax_ignore, spans=SL[ES[Span(start=1, end=6, text='12.05'),
Span(start=7, end=8, text='-'),
Span(start=9, end=14, text='12.35'),
Span(start=15, end=16, text='"'),
Span(start=16, end=20, text='Õnne'),
Span(start=21, end=23, text='13'),
Span(start=23, end=24, text='"'),
Span(start=25, end=26, text='('),
Span(start=26, end=28, text='1.'),
Span(start=29, end=32, text='osa'),
Span(start=32, end=33, text=')'),
Span(start=34, end=39, text='12.35'),
Span(start=40, end=41, text='-'),
Span(start=42, end=47, text='13.05'),
Span(start=48, end=49, text='"'),
Span(start=49, end=53, text='Õnne'),
Span(start=54, end=56, text='13'),
Span(start=56, end=57, text='"'),
Span(start=58, end=59, text='('),
Span(start=59, end=61, text='1.'),
Span(start=62, end=65, text='osa'),
Span(start=66, end=72, text='kordus'),
Span(start=72, end=73, text=')'),
Span(start=74, end=79, text='13.05'),
Span(start=80, end=81, text='-'),
Span(start=82, end=87, text='13.35'),
Span(start=88, end=89, text='"'),
Span(start=89, end=93, text='Õnne'),
Span(start=94, end=96, text='13'),
Span(start=96, end=97, text='"'),
Span(start=98, end=99, text='('),
Span(start=99, end=101, text='2.'),
Span(start=102, end=105, text='osa'),
Span(start=105, end=106, text=')')]])

#### Flag `ignore_sentences_with_comma_separated_num_name_lists`

Sentences containing comma separated list of titlecase words / numbers ( like sport results, player/country listings, game scores etc.) will be ignored.

In [9]:
# Create text and preprocess
from estnltk.text import Text
text = Text('''
Veerandfinaalid .
Lindsay Davenport ( 6 ) , USA-Jana Novotna ( 3 ) , Tšehhi 6 : 2 , 4 : 6 , 7 : 6 , Martina Hingis ( 1 ) , Šveits-Arantxa Sanchez Vicario ( 10 ) , Hispaania 6 : 3 , 6 : 2 .
Paarismängu veerandfinaalid .
Gigi Fernandez , USA/Nataša Zvereva , Valgevene- Alexandra Fusai/Nathalie Tauziat , Prantsusmaa 4 : 6 , 6 : 2 , 6 : 2 , Nicole Arendt , USA/Manon Bollegraf , Holland-Ruxandra Dragomir , Rumeenia/Iva Majoli , Horvaatia 6 : 3 , 3 : 6 , 6 : 4 .
''')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_sentences_with_comma_separated_num_name_lists=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name=syntax_ignore, spans=SL[ES[Span(start=19, end=26, text='Lindsay'),
Span(start=27, end=36, text='Davenport'),
Span(start=37, end=38, text='('),
Span(start=39, end=40, text='6'),
Span(start=41, end=42, text=')'),
Span(start=43, end=44, text=','),
Span(start=45, end=53, text='USA-Jana'),
Span(start=54, end=61, text='Novotna'),
Span(start=62, end=63, text='('),
Span(start=64, end=65, text='3'),
Span(start=66, end=67, text=')'),
Span(start=68, end=69, text=','),
Span(start=70, end=76, text='Tšehhi'),
Span(start=77, end=78, text='6'),
Span(start=79, end=80, text=':'),
Span(start=81, end=86, text='2 , 4'),
Span(start=87, end=88, text=':'),
Span(start=89, end=94, text='6 , 7'),
Span(start=95, end=96, text=':'),
Span(start=97, end=98, text='6'),
Span(start=99, end=100, text=','),
Span(start=101, end=108, text='Martina'),
Span(start=109, end=115, text='Hingis'),
Span(start=116, end=117, text='('),
Span(start=118, end=119, text='1'),
Span(start=120, end=121, text=')'),
Span(start=122, end=123, text=','),
Span(start=124, end=138, text='Šveits-Arantxa'),
Span(start=139, end=146, text='Sanchez'),
Span(start=147, end=154, text='Vicario'),
Span(start=155, end=156, text='('),
Span(start=157, end=159, text='10'),
Span(start=160, end=161, text=')'),
Span(start=162, end=163, text=','),
Span(start=164, end=173, text='Hispaania'),
Span(start=174, end=175, text='6'),
Span(start=176, end=177, text=':'),
Span(start=178, end=183, text='3 , 6'),
Span(start=184, end=185, text=':'),
Span(start=186, end=189, text='2 .')],
ES[Span(start=220, end=224, text='Gigi'),
Span(start=225, end=234, text='Fernandez'),
Span(start=235, end=236, text=','),
Span(start=237, end=240, text='USA'),
Span(start=240, end=241, text='/'),
Span(start=241, end=247, text='Nataša'),
Span(start=248, end=255, text='Zvereva'),
Span(start=256, end=257, text=','),
Span(start=258, end=268, text='Valgevene-'),
Span(start=269, end=278, text='Alexandra'),
Span(start=279, end=284, text='Fusai'),
Span(start=284, end=285, text='/'),
Span(start=285, end=293, text='Nathalie'),
Span(start=294, end=301, text='Tauziat'),
Span(start=302, end=303, text=','),
Span(start=304, end=315, text='Prantsusmaa'),
Span(start=316, end=317, text='4'),
Span(start=318, end=319, text=':'),
Span(start=320, end=325, text='6 , 6'),
Span(start=326, end=327, text=':'),
Span(start=328, end=333, text='2 , 6'),
Span(start=334, end=335, text=':'),
Span(start=336, end=337, text='2'),
Span(start=338, end=339, text=','),
Span(start=340, end=346, text='Nicole'),
Span(start=347, end=353, text='Arendt'),
Span(start=354, end=355, text=','),
Span(start=356, end=359, text='USA'),
Span(start=359, end=360, text='/'),
Span(start=360, end=365, text='Manon'),
Span(start=366, end=375, text='Bollegraf'),
Span(start=376, end=377, text=','),
Span(start=378, end=394, text='Holland-Ruxandra'),
Span(start=395, end=403, text='Dragomir'),
Span(start=404, end=405, text=','),
Span(start=406, end=414, text='Rumeenia'),
Span(start=414, end=415, text='/'),
Span(start=415, end=418, text='Iva'),
Span(start=419, end=425, text='Majoli'),
Span(start=426, end=427, text=','),
Span(start=428, end=437, text='Horvaatia'),
Span(start=438, end=439, text='6'),
Span(start=440, end=441, text=':'),
Span(start=442, end=447, text='3 , 3'),
Span(start=448, end=449, text=':'),
Span(start=450, end=455, text='6 , 6'),
Span(start=456, end=457, text=':'),
Span(start=458, end=461, text='4 .')]])

#### Flag  `ignore_sentences_consisting_of_numbers`

Detects sentences that only contain number or numbers, no letters, and do not end with '!' nor '?'.

### Patterns for ignoring groups of consecutive sentences

#### Flag  `ignore_consecutive_parenthesized_sentences `

If consecutive sentences all contain parenthesized content that is already ignored, and all of these sentences contain less than 3 consecutive lowercase words, then these sentences likely represent enumerations (e.g. sports results) which can be safely ignored.

In [10]:
# Create text and preprocess
from estnltk.text import Text
text = Text('''Eile õhtul sõidetud avakatsel sai Markko Märtin ( Subaru , pildil ) viienda aja .
Tulemused :
1. Tommi Mäkinen ( FIN ) Mitsubishi - 3.46 , 9
2. Marcus Grönholm ( FIN ) Peugeot +1,0
3. Harri Rovanperä ( FIN ) Peugeot +4,1
4. Carlos Sainz ( ESP ) Ford +6,0''')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_consecutive_parenthesized_sentences=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name=syntax_ignore, spans=SL[ES[Span(start=82, end=91, text='Tulemused'),
Span(start=92, end=93, text=':'),
Span(start=94, end=96, text='1.')],
ES[Span(start=97, end=102, text='Tommi'),
Span(start=103, end=110, text='Mäkinen'),
Span(start=111, end=112, text='('),
Span(start=113, end=116, text='FIN'),
Span(start=117, end=118, text=')'),
Span(start=119, end=129, text='Mitsubishi'),
Span(start=130, end=131, text='-'),
Span(start=132, end=140, text='3.46 , 9'),
Span(start=141, end=143, text='2.')],
ES[Span(start=144, end=150, text='Marcus'),
Span(start=151, end=159, text='Grönholm'),
Span(start=160, end=161, text='('),
Span(start=162, end=165, text='FIN'),
Span(start=166, end=167, text=')'),
Span(start=168, end=175, text='Peugeot'),
Span(start=176, end=180, text='+1,0'),
Span(start=181, end=183, text='3.')],
ES[Span(start=184, end=189, text='Harri'),
Span(start=190, end=199, text='Rovanperä'),
Span(start=200, end=201, text='('),
Span(start=202, end=205, text='FIN'),
Span(start=206, end=207, text=')'),
Span(start=208, end=215, text='Peugeot'),
Span(start=216, end=220, text='+4,1'),
Span(start=221, end=223, text='4.')],
ES[Span(start=224, end=230, text='Carlos'),
Span(start=231, end=236, text='Sainz'),
Span(start=237, end=238, text='('),
Span(start=239, end=242, text='ESP'),
Span(start=243, end=244, text=')'),
Span(start=245, end=249, text='Ford'),
Span(start=250, end=254, text='+6,0')]])

#### Flag  `ignore_consecutive_enum_ucase_num_sentences`

Detects sentences that: 1) start with an uppercase letter, or an ordinal number followed by an uppercase letter, or an ordinal number; 2) contain at least one number; 3) does not contain 3 consecutive lowercase words; 4) are a part of at least 4 consecutive sentences that have the same properties (1, 2 and 3).

In [11]:
# Create text and preprocess
from estnltk.text import Text
text = Text('''Maakonniti summeerides on tulumaksu laekumise viis esimest :
1. Harjumaa 2792 ,
2. Hiiumaa 2119 ,
3. Tartumaa 2081 ,
4. Läänemaa 1933 ,
5. Pärnumaa 1903.''')
text.tag_layer(['words', 'sentences'])

# Apply syntax_ignore_tagger
syntax_ignore_tagger = SyntaxIgnoreTagger(ignore_consecutive_enum_ucase_num_sentences=True)
syntax_ignore_tagger.tag(text)

# Examine results
text.syntax_ignore

Layer(name=syntax_ignore, spans=SL[ES[Span(start=64, end=72, text='Harjumaa'),
Span(start=73, end=77, text='2792'),
Span(start=78, end=79, text=','),
Span(start=80, end=82, text='2.')],
ES[Span(start=83, end=90, text='Hiiumaa'),
Span(start=91, end=95, text='2119'),
Span(start=96, end=97, text=','),
Span(start=98, end=100, text='3.')],
ES[Span(start=101, end=109, text='Tartumaa'),
Span(start=110, end=114, text='2081'),
Span(start=115, end=116, text=','),
Span(start=117, end=119, text='4.')],
ES[Span(start=120, end=128, text='Läänemaa'),
Span(start=129, end=133, text='1933'),
Span(start=134, end=135, text=','),
Span(start=136, end=138, text='5.')],
ES[Span(start=139, end=147, text='Pärnumaa'),
Span(start=148, end=153, text='1903.')]])

## Integration with the syntactic analysis pre-processing module

Integration with the syntactic analysis pre-processing module is yet to be implemented.